In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install -r requirements.txt


Explore stock market dataset from Yahoo Finance

In [ ]:
import yfinance as yf
import pandas as pd


## Prepare broad market indicies

In [ ]:
# Capture S&P500, NASDAQ100 and Russell 200 indecies and their equal weighted counter parts
# As well as VIX volatility index, DYX US Dollar index and TNX US 10 Year Treasuries Rate Index
broad_market_indicies = '^SPX ^SPXEW ^NDX ^NDXE ^RUT ^R2ESC ^VIX DX-Y.NYB ^TNX'

In [ ]:
broad_market = yf.download(broad_market_indicies, period='max', group_by='tickers') 
broad_market

In [ ]:
broad_market.to_csv('data/broad_market.csv.bz2', index='Date')

## Prepare Sector Indicies

In [ ]:
sector_indicies = 'XLE ^SP500-15 ^SP500-20 ^SP500-25 ^SP500-30 ^SP500-35 ^SP500-40 ^SP500-45 ^SP500-50 ^SP500-55 ^SP500-60'

In [ ]:
sectors = yf.download(sector_indicies, period='max') 
sectors

In [ ]:
sectors.to_csv('data/sectors.csv.bz2')

## Load list of IBD Growth Stocks

In [8]:

ibd50 = pd.read_csv('data/IBD50.csv')
ibd50


,Symbol
0,GCT
1,NVDA
2,CRWD
3,RCL
4,AMPH
5,ELF
6,CCL
7,CELH
8,ASND
9,ZS


In [9]:
ibd250 = pd.read_csv('data/IBD250.csv')
ibd250

,Symbol
0,INTU
1,ALKT
2,FICO
3,GWRE
4,COIN
...,...
295,CAMT
296,STLA
297,RACE
298,RYAAY


In [10]:
# merge IBD 50 and 250 symbol sets
ibd50_set = set(ibd50['Symbol'])
ibd250_set = set(ibd250['Symbol'])
ibd_growth_set = ibd50_set.union(ibd250_set)

In [11]:
len(ibd_growth_set)

309

In [12]:
ibdgrowth_str = ' '.join(ibd_growth_set)
ibdgrowth_str



'SSD TPH LEN FERG ZG TEAM MEDP LULU ASO FLT FBIN FLEX SCCO ELF VSTS AEO OC ROST LRN AZEK JPM BK RACE TRI YELP CRS MCO ZS CADE CELH DUOL WOR PSN CSWC GE CCL TREX TRIP RELX NFLX VRNS MAIN WAL MFC TSLX FITB STRL PTC LPG OZK HWKN HWM ETN SPOT ERIE HG MBWM SFM TOL CAT ICE WDAY BROS BCO GPS PVH DOCU IBP VRTX QLYS AZTA PH REGN Z CRBG ALLY TDW JBI RCL EXPE ALKT TMUS TBBK OFG ALKS AOS ARM CYBR CHKP WBS TGH KBH APO SKX KKR RAMP DHI AWI BLX FOUR COLB KNF WTFC MAS FI MARA PCOR NBIX RGA GOOGL BKNG TWLO UBS ZTS ITCI QCOM ASND LHX FICO FRPT MMYT AMZN INFA IMGN URBN FTAI GOOG DFH PBRA INTU BAC CDRE GBDC DV CRH GEN MBIN FOR DB EME FNF CAMT NMIH AMK DT PLUS RY LMB RL STLA OSW PCAR GVA AER FSS NTNX ERJ IHG TMHC BLD PATH DXCM CRSP MATX DOCN SMCI BR ALTR FIX GPN SHW COIN ARCO IOT AX WFC CAH AXP COST AGO S GPK DBX NVDA URI RYAAY XP PLAB PANW IT AMP SNOW V STNE PINS BX CMG AVGO CRWD AXON TWST ITT OTEX FRSH ENSG ANF WIX WAB PWR PDD MNST GDDY PRI ASML GWRE MSFT SPXC FLR ABBNY OWL ISRG MLM WRB CLS ROL HOLI WSM 

In [14]:
stocks_ticker_set = ibd_growth_set

## Prepare stocks price data

In [ ]:
ibdgrowth_data = yf.download(ibdgrowth_str, period='max', group_by='tickers') 
ibdgrowth_data

In [ ]:
ibdgrowth_data.columns.levels

In [ ]:
ibdgrowth_data.to_csv('data/ibdgrowth_hist.csv.bz2', index='Date')

In [20]:
ibdgrowth_loaded = pd.read_csv('data/ibdgrowth_hist.csv.bz2', header=[0, 1], index_col=0)
ibdgrowth_loaded

TWST                                              \
                          Open       High        Low      Close  Adj Close   
Date                                                                         
1962-01-02 00:00:00        NaN        NaN        NaN        NaN        NaN   
1962-01-03 00:00:00        NaN        NaN        NaN        NaN        NaN   
1962-01-04 00:00:00        NaN        NaN        NaN        NaN        NaN   
1962-01-05 00:00:00        NaN        NaN        NaN        NaN        NaN   
1962-01-08 00:00:00        NaN        NaN        NaN        NaN        NaN   
...                        ...        ...        ...        ...        ...   
2024-01-08 00:00:00  33.830002  35.070000  32.799999  34.980000  34.980000   
2024-01-09 00:00:00  34.250000  37.520000  34.130001  37.150002  37.150002   
2024-01-10 00:00:00  37.080002  37.410000  35.250000  36.580002  36.580002   
2024-01-11 00:00:00  35.959999  36.259998  34.180000  35.840000  35.840000   
2024-01-12 00:00:00  36.029999  37.730000  35.919998  36.119999  36.119999   

                                      NFLX                          \
                        Volume        Open        High         Low   
Date                                                                 
1962-01-02 00:00:00        NaN         NaN         NaN         NaN   
1962-01-03 00:00:00        NaN         NaN         NaN         NaN   
1962-01-04 00:00:00        NaN         NaN         NaN         NaN   
1962-01-05 00:00:00        NaN         NaN         NaN         NaN   
1962-01-08 00:00:00        NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
2024-01-08 00:00:00   943300.0  473.890015  485.239990  473.649994   
2024-01-09 00:00:00  1081500.0  475.529999  485.089996  473.799988   
2024-01-10 00:00:00  1424400.0  481.600006  487.000000  472.950012   
2024-01-11 00:00:00  1188700.0  492.540009  503.410004  483.820007   
2024-01-12 00:00:00   896500.0  500.510010  500.790009  490.609985   

                                 ...       FITB                        \
                          Close  ...        Low      Close  Adj Close   
Date                             ...                                    
1962-01-02 00:00:00         NaN  ...        NaN        NaN        NaN   
1962-01-03 00:00:00         NaN  ...        NaN        NaN        NaN   
1962-01-04 00:00:00         NaN  ...        NaN        NaN        NaN   
1962-01-05 00:00:00         NaN  ...        NaN        NaN        NaN   
1962-01-08 00:00:00         NaN  ...        NaN        NaN        NaN   
...                         ...  ...        ...        ...        ...   
2024-01-08 00:00:00  485.029999  ...  34.900002  35.639999  35.639999   
2024-01-09 00:00:00  482.089996  ...  34.950001  35.310001  35.310001   
2024-01-10 00:00:00  478.329987  ...  34.919998  35.360001  35.360001   
2024-01-11 00:00:00  492.230011  ...  34.310001  34.849998  34.849998   
2024-01-12 00:00:00  492.160004  ...  34.090000  34.360001  34.360001   

                                       PNC                          \
                        Volume        Open        High         Low   
Date                                                                 
1962-01-02 00:00:00        NaN         NaN         NaN         NaN   
1962-01-03 00:00:00        NaN         NaN         NaN         NaN   
1962-01-04 00:00:00        NaN         NaN         NaN         NaN   
1962-01-05 00:00:00        NaN         NaN         NaN         NaN   
1962-01-08 00:00:00        NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
2024-01-08 00:00:00  4698900.0  156.000000  157.690002  154.580002   
2024-01-09 00:00:00  4248200.0  156.220001  156.220001  154.009995   
2024-01-10 00:00:00  3747900.0  154.500000  155.419998  153.210007   
2024-01-11 00:00:00  6155900.0  154.529999  154.539993  149.880005   
2024-01-12 00:00:00

In [ ]:
for ticker in ibdgrowth_loaded.columns.levels[0][:2]:
    print(f'ticker: {ticker}')
    ticker_data = ibdgrowth_loaded[ticker]
    print(f'ticker historic data: {ticker_data}')
    # remove missing values
    ticker_data = ticker_data.dropna()
    print(f'ticker historic data without missing data: {ticker_data}')


## Prepare historical stock sales and earnings data

In [19]:
from dotenv import load_dotenv
import os

load_dotenv()

FMP_API_KEY=os.getenv("FMP_API_KEY")

print(f'FMP_API_KEY={FMP_API_KEY}')

FMP_API_KEY=pkq8EGvu9HaixuEK8vSSxbC62JrlXmEp


In [18]:
import fmpsdk

# Company Valuation Methods
symbol: str = "AAPL"
symbols: ["AAPL", "CSCO", "QQQQ"]
exchange: str = "NYSE"
exchanges: ["NYSE", "NASDAQ"]
query: str = "AA"
limit: int = 3
period: str = "quarter"
download: bool = True
market_cap_more_than: int = 1000000000
beta_more_than: int = 1
volume_more_than: int = 10000
sector: str = "Technology"
dividend_more_than: int = 0
industry: str = "Software"
filing_type: str = "10-K"
print(f"Company Profile: {fmpsdk.company_profile(apikey=FMP_API_KEY, symbol=symbol)=}")


Company Profile: fmpsdk.company_profile(apikey=FMP_API_KEY, symbol=symbol)=[{'symbol': 'AAPL', 'price': 185.92, 'beta': 1.29, 'volAvg': 54148509, 'mktCap': 2891576576000, 'lastDiv': 0.96, 'range': '131.66-199.62', 'changes': 0.33, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchange': 'NASDAQ Global Select', 'exchangeShortName': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to disco

In [ ]:
# there should be no duplicate symbols in this list
assert not ibdgrowth_loaded.columns.levels[0].duplicated().any() 

In [ ]:
earnings_all_df = pd.DataFrame()
for ticker in stocks_ticker_set:
    earnings = fmpsdk.historical_earning_calendar(apikey=FMP_API_KEY, symbol=ticker, limit=-1)
    if earnings is not None and len(earnings) > 0:
        edf = pd.DataFrame(earnings)
        edf['date'] = pd.to_datetime(edf['date'])
        edf = edf.set_index(['symbol', 'date'])
        print(f"Earnings calendar for {ticker}: \n{edf}")
        # edf = edf.pivot(columns='symbol')
        # edf.swaplevel(i=0,j=1, axis=0)
        # edf.drop(columns=['symbol'])
        earnings_all_df = pd.concat([earnings_all_df, edf])
        print(f"Earnings calendar after pivot for {ticker}: \n{edf}")
        n_earnings = len(earnings)
        print(f"Total earnings reports for {ticker}: {n_earnings}")
#    earliest_earn = earnings[-1] if len(earnings > 0 else 'None')
#    print(f"Earliest earnings report for {ticker}: {earliest_earn}")


In [ ]:
earnings

In [ ]:
aaon = earnings_all_df.loc[['AAON']]

In [ ]:
aaon

In [ ]:
len(earnings_all_df)

In [ ]:
earnings_all_df


In [ ]:
earnings_file = 'data/earnings_calendar.csv.bz2'

In [ ]:
earnings_all_df.to_csv(earnings_file)

### Read back data and verify it

In [ ]:
import pandas as pd

earnings_loaded_df = pd.read_csv('data/earnings_calendar.csv.bz2', index_col=['symbol', 'date'])
print(earnings_loaded_df)

## Prepare key metrics data for company fundamentals

In [22]:
keymetrics_all_df = pd.DataFrame()
for ticker in stocks_ticker_set:
    kms = fmpsdk.key_metrics(apikey=FMP_API_KEY, symbol=ticker, period='quarter', limit=-1)
    if kms is not None and len(kms) > 0:
        kms_df = pd.DataFrame(kms)
        kms_df['date'] = pd.to_datetime(kms_df['date'])
        kms_df = kms_df.set_index(['symbol', 'date'])
        print(f"Key metrics for {ticker} sample: \n{kms_df.columns}")
        keymetrics_all_df = pd.concat([keymetrics_all_df, kms_df])
        print(f"Key metrics concatenated {ticker}: \n{keymetrics_all_df.columns}")
        n_kms = len(kms_df)
        print(f"Total key metrics reports for {ticker}: {n_kms}")


Key metrics for SSD sample: 
Index(['calendarYear', 'period', 'revenuePerShare', 'netIncomePerShare',
       'operatingCashFlowPerShare', 'freeCashFlowPerShare', 'cashPerShare',
       'bookValuePerShare', 'tangibleBookValuePerShare',
       'shareholdersEquityPerShare', 'interestDebtPerShare', 'marketCap',
       'enterpriseValue', 'peRatio', 'priceToSalesRatio', 'pocfratio',
       'pfcfRatio', 'pbRatio', 'ptbRatio', 'evToSales',
       'enterpriseValueOverEBITDA', 'evToOperatingCashFlow',
       'evToFreeCashFlow', 'earningsYield', 'freeCashFlowYield',
       'debtToEquity', 'debtToAssets', 'netDebtToEBITDA', 'currentRatio',
       'interestCoverage', 'incomeQuality', 'dividendYield', 'payoutRatio',
       'salesGeneralAndAdministrativeToRevenue',
       'researchAndDdevelopementToRevenue', 'intangiblesToTotalAssets',
       'capexToOperatingCashFlow', 'capexToRevenue', 'capexToDepreciation',
       'stockBasedCompensationToRevenue', 'grahamNumber', 'roic',
       'returnOnTangibleA

Experiment with other stock data

In [23]:
keymetrics_all_df

calendarYear period  revenuePerShare  netIncomePerShare  \
symbol date                                                                 
SSD    2023-09-30         2023     Q3        13.593701           2.437630   
       2023-06-30         2023     Q2        14.005015           2.512620   
       2023-03-31         2023     Q1        12.542361           2.064163   
       2022-12-31         2022     Q4        11.172179           1.353119   
       2022-09-30         2022     Q3        12.932100           2.061126   
...                        ...    ...              ...                ...   
ARVN   2018-03-31         2018     Q1         0.198178          -0.200194   
       2017-12-31         2017     Q4         0.124075          -0.228056   
       2017-09-30         2017     Q3         0.080497          -0.308572   
       2017-06-30         2017     Q2         0.879481          -3.519372   
       2017-03-31         2017     Q1         0.879481          -3.291460   

                   operatingCashFlowPerShare  freeCashFlowPerShare  \
symbol date                                                          
SSD    2023-09-30                   0.000000              0.000000   
       2023-06-30                   4.553235              4.104197   
       2023-03-31                   0.069420             -0.370805   
       2022-12-31                   3.204571              2.602015   
       2022-09-30                   2.918389              2.690842   
...                                      ...                   ...   
ARVN   2018-03-31                   0.930897              0.900092   
       2017-12-31                   1.358169              1.337208   
       2017-09-30                  -0.358904             -0.358676   
       2017-06-30                  -4.494133             -4.544222   
       2017-03-31                  -3.728831             -3.985769   

                   cashPerShare  bookValuePerShare  tangibleBookValuePerShare  \
symbol date                                                                     
SSD    2023-09-30     13.380967          39.122911                  19.441544   
       2023-06-30      9.561555          37.013101                  16.815604   
       2023-03-31      5.926801          34.907510                  14.563201   
       2022-12-31     14.128629          33.370008                  13.013107   
       2022-09-30      7.223554          31.241796                  12.590381   
...                         ...                ...                        ...   
ARVN   2018-03-31      5.435910           2.480791                   2.480791   
       2017-12-31      1.889431           0.011860                   0.011860   
       2017-09-30           NaN                NaN                        NaN   
       2017-06-30           NaN                NaN                        NaN   
       2017-03-31     18.738486          12.694176                   0.000000   

                   shareholdersEquityPerShare  ...  averagePayables  \
symbol date                                    ...                    
SSD    2023-09-30                   39.122911  ...       96557000.0   
       2023-06-30                   37.013101  ...       96574500.0   
       2023-03-31                   34.907510  ...       96571500.0   
       2022-12-31                   33.370008  ...       98243500.0   
       2022-09-30                   31.241796  ...      105807000.0   
...                                       ...  ...              ...   
ARVN   2018-03-31                    2.480791  ...        1281630.5   
       2017-12-31                    0.011860  ...              NaN   
       2017-09-30                         NaN  ...              NaN   
       2017-06-30                         NaN  ...              NaN   
       2017-03-31                   12.694176  ...              NaN   

                   averageInventory  daysSalesOutstanding  \
symbol date                                                 
SSD  

In [24]:
kms_file = 'data/keymetrics_history.csv.bz2'
keymetrics_all_df.to_csv(kms_file)

## Prepare forward looking analyst estimates to be used as future covariates

In [41]:
DEFAULT_LIMIT=-1
import typing
from fmpsdk.url_methods import __return_json_v3, __validate_period


def analyst_estimates(
    apikey: str, 
    symbol: str, 
    period: str = "annual",
    limit: int = DEFAULT_LIMIT
) -> typing.Optional[typing.List[typing.Dict]]:
    """
    Query FMP /analyst-estimates/ API.

    :param apikey: Your API key.
    :param symbol: Company ticker.
    :param period: 'annual' or 'quarter'
    :param limit: Number of rows to return.
    :return: A list of dictionaries.
    """
    path = f"/analyst-estimates/{symbol}"
    query_vars = {
        "apikey": apikey,
        "symbol": symbol,
        "period": __validate_period(value=period),
        "limit": limit,
    }
    return __return_json_v3(path=path, query_vars=query_vars)



In [46]:

def fetch_estimates(period=None):
    assert period in ['quarter', 'annual']
    estimates_all_df = pd.DataFrame()
    for ticker in stocks_ticker_set:
        est = analyst_estimates(apikey=FMP_API_KEY, symbol=ticker, period=period, limit=-1)
        # print('est:', est)
        if est is not None and len(est) > 0:
            est_df = pd.DataFrame(est)
            est_df['date'] = pd.to_datetime(est_df['date'])
            est_df = est_df.set_index(['symbol', 'date'])
            # print(f"Analyst estimates for {ticker} sample: \n{est_df.columns}")
            estimates_all_df = pd.concat([estimates_all_df, est_df])
            # print(f"Key metrics concatenated {ticker}: \n{estimates_all_df.columns}")
            n_est = len(est_df)
            print(f"Total estimates reports for {ticker}: {n_est}")
    return estimates_all_df



In [47]:
for p in ['annual', 'quarter']:
    estimates_all_df = fetch_estimates(p)
    est_file_name= f'data/analyst_estimates_{p}.csv.bz2'    
    estimates_all_df.to_csv(est_file_name)
    print(f'{p} estimates:', estimates_all_df)
    

Total estimates reports for SSD: 30
Total estimates reports for TPH: 14
Total estimates reports for LEN: 30
Total estimates reports for FERG: 5
Total estimates reports for ZG: 15
Total estimates reports for TEAM: 11
Total estimates reports for MEDP: 10
Total estimates reports for LULU: 20
Total estimates reports for ASO: 6
Total estimates reports for FLT: 16
Total estimates reports for FBIN: 15
Total estimates reports for FLEX: 31
Total estimates reports for SCCO: 28
Total estimates reports for ELF: 10
Total estimates reports for AEO: 30
Total estimates reports for OC: 28
Total estimates reports for ROST: 30
Total estimates reports for LRN: 20
Total estimates reports for JPM: 29
Total estimates reports for BK: 30
Total estimates reports for RACE: 11
Total estimates reports for TRI: 30
Total estimates reports for YELP: 14
Total estimates reports for CRS: 31
Total estimates reports for MCO: 26
Total estimates reports for ZS: 9
Total estimates reports for CADE: 9
Total estimates reports f

ERROR:root:Connection to https://financialmodelingprep.com/api/v3//analyst-estimates/MS failed:  DNS failure, refused connection or some other connection related issue.


Total estimates reports for CNM: 14
Total estimates reports for PNC: 112
Total estimates reports for GTLB: 12
Total estimates reports for SHOP: 36
Total estimates reports for STN: 86
Total estimates reports for WING: 36
Total estimates reports for NET: 21
Total estimates reports for IDXX: 112
Total estimates reports for DASH: 17
Total estimates reports for BOH: 112
Total estimates reports for IR: 30
Total estimates reports for GEL: 77
Total estimates reports for ANET: 42
Total estimates reports for AROC: 91
Total estimates reports for TXRH: 80
Total estimates reports for MNDY: 14
Total estimates reports for MOD: 108
Total estimates reports for ADSK: 113
Total estimates reports for AMGN: 112
Total estimates reports for XYL: 53
Total estimates reports for COF: 112
Total estimates reports for NTRA: 34
Total estimates reports for AFYA: 20
Total estimates reports for DKS: 88
Total estimates reports for SKWD: 6
Total estimates reports for BLBD: 34
Total estimates reports for AMPH: 31
Total e